In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import sys
import os
sys.path.append(os.path.abspath("../.."))  # Adds the project root to sys.path
import csv
import numpy as np 
import tensorflow as tf
from tensorflow import keras

from recommenders.datasets.linear_regression import save_rating_values
from recommenders.datasets.linear_regression import get_data
from recommenders.datasets.linear_regression import cofi_cost_func_v
from recommenders.datasets.linear_regression import normalize_ratings
from recommenders.datasets.linear_regression import load_csv_np
from recommenders.datasets.linear_regression import load_csv_list

In [9]:
save_rating_values()

In [2]:
X, Y, R, num_features, num_products = get_data()

Y (7, 3) R (7, 3)
X (7, 6)
num_features 6
num_products 7
num_users 3


In [3]:
my_ratings = np.zeros(num_products)

my_ratings[0] = 5 # Bosch, Burgija
my_ratings[3] = 2 # Makita, Secica

# Add new user ratings to Y 
Y = np.c_[my_ratings, Y]

# Add new user indicator matrix to R
R = np.c_[(my_ratings != 0).astype(int), R]

# Normalize the Dataset
Ynorm, Ymean = normalize_ratings(Y, R)
print(Ynorm)
print(Ymean)

[[ 2.  0. -2.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [-2.  1.  1.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]]
[[3. ]
 [5. ]
 [5. ]
 [4. ]
 [4. ]
 [3.5]
 [3.5]]


In [4]:
print(Ymean.shape)

(7, 1)


In [5]:
num_users = Y.shape[1] # FT: It's mandatory because we added one more user to the matrix

# Set Initial Parameters (W, X), use tf.Variable to track these variables
tf.random.set_seed(1234) # for consistent results
W = tf.Variable(tf.random.normal((num_users,  num_features),dtype=tf.float64),  name='W')
X = tf.Variable(tf.random.normal((num_products, num_features),dtype=tf.float64),  name='X')
b = tf.Variable(tf.random.normal((1,          num_users),   dtype=tf.float64),  name='b')

# Instantiate an optimizer.
optimizer = keras.optimizers.Adam(learning_rate=1e-1)

In [6]:
iterations = 10
lambda_ = 1

for iter in range(iterations):
    # Use TensorFlow’s GradientTape
    # to record the operations used to compute the cost 
    with tf.GradientTape() as tape:

        # Compute the cost (forward pass included in cost)
        cost_value = cofi_cost_func_v(X, W, b, Ynorm, R, lambda_)

    # Use the gradient tape to automatically retrieve
    # the gradients of the trainable variables with respect to the loss
    grads = tape.gradient( cost_value, [X,W,b] )

    # Run one step of gradient descent by updating
    # the value of the variables to minimize the loss.
    optimizer.apply_gradients( zip(grads, [X,W,b]) )

    # Log periodically.
    if iter % 20 == 0:
        print(f"Training loss at iteration {iter}: {cost_value:0.1f}")

Training loss at iteration 0: 102.4


In [7]:
product_names = load_csv_list('../../recommenders/datasets/pa/product_names.csv')
print(product_names)

# Make a prediction using trained weights and biases
p = np.matmul(X.numpy(), np.transpose(W.numpy())) + b.numpy()

#restore the mean
pm = p + Ymean

my_predictions = pm[:,0]

# sort predictions
ix = tf.argsort(my_predictions, direction='DESCENDING')

print('\n\nOriginal vs Predicted ratings:\n')
for i in range(len(my_ratings)):
    # print(f'{my_predictions[i]:0.2f}')
    print(f'{my_predictions[i]}')
    # if my_ratings[i] > 0:
    #     print(f'Original {my_ratings[i]}, Predicted {my_predictions[i]:0.2f} for {product_names[i]}')

[['Bosch GSB 120'], ['Bosch GSH 240'], ['Bosch GSH 360'], ['DeWalt DE 14'], ['DeWalt DE 15'], ['Makita MA 00A'], ['Makita MA 00B']]


Original vs Predicted ratings:

5.892576987276932
5.282840283321487
3.3996140434329676
2.7532721113116585
4.74877957385668
4.036539790793527
1.919480123082662
